In [4]:
import pickle
import os

import pandas as pd
import numpy as np
from scipy import stats
from sklearn import metrics
from sklearn import model_selection
from gpmodel import gpmodel, gpkernel, chimera_tools

In [5]:
with open('../inputs/props.pkl', 'rb') as f:
    df = pickle.load(f)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../inputs/props.pkl'

NameError: name 'df' is not defined

In [3]:
def select_X_and_Y(df, all_X, y_column):
    not_dropped = ~pd.isnull(df[y_column])
    not_dropped = pd.Series(not_dropped, index=df.index)
    Ys = df[not_dropped][y_column]
    gens = df[not_dropped]['generation']
    Ys.index = df[not_dropped]['name']
    Xs = all_X.loc[Ys.index]
    return Xs, Ys, gens

In [4]:
tasks = ['bin0.1_max_peak', '']
lits = [False]
mtypes = [gpmodel.GPClassifier]

In [5]:
with open('../inputs/X_and_terms.pkl', 'rb') as f:
    X_all, terms = pickle.load(f)

In [6]:
def train_and_save(df, task, fname, mtype, guesses=None):
    X, y, _ = select_X_and_Y(df, X_all, task)
    X = X.values
    y = y.values  
    k = gpkernel.MaternKernel('5/2')
    clf = mtype(k, guesses=guesses)
    clf.fit(X, y)
    clf.dump(fname)
    return clf

for task, lit, mtype in zip(tasks, lits, mtypes):
    fname = '../outputs/matern52_' + task + '_' + str(lit) + '.pkl'
    if lit:
        clf = train_and_save(df, task, fname, mtype)
    else:
        clf = train_and_save(df[df['generation'] != 8], task, fname, mtype)
    print(fname)
    print(clf.hypers, clf.ML)

../outputs/matern52_bin0.1_max_peak_False.pkl
[ 20.88092844] [ 76.59353427]


In [7]:
cls_dict = {True:gpmodel.GPClassifier.load, False: gpmodel.GPRegressor.load}
clfs = [cls_dict['bin' in path]('../outputs/' + path) for path in os.listdir('../outputs/') if path != '.DS_Store']
fnames = ['.'.join(path.split('.')[:-1]) for path in os.listdir('../outputs/') if path != '.DS_Store']
fnames

['matern52_bin0.1_max_peak_False']

In [8]:
for fname in fnames:
    with open('../outputs/' + fname + '.txt', 'w') as f:
        if 'bin' in fname:
            f.write('name,p,mu,var\n')
        else:
            f.write('name,mu,var\n')

In [9]:
%%time
df = pd.read_csv('../inputs/all_lit_chimeras_gaps.txt', index_col=0)
with open('../inputs/lit_alignment_and_contacts.pkl', 'rb') as f:
    ss, contacts = pickle.load(f)
amino_acids = ('G', 'A', 'L', 'M', 'F', 'W', 'K', 'Q', 'E', 'S',
       'P', 'V', 'I', 'C', 'Y', 'H', 'R', 'N', 'D', 'T', '-')
sample_space = [amino_acids for _ in ss]
n_splits = 1000
n_per = len(df.index) // n_splits
inds = [df.index[n * n_per: (n+1) * n_per]
        for n in range(n_splits)]
inds.append(df.index[n_splits * n_per::])
seq_terms = chimera_tools.make_sequence_terms(sample_space)
struct_terms = chimera_tools.contacting_terms(sample_space, contacts)
all_terms = seq_terms + struct_terms

for i, ind in enumerate(inds):
    seqs = df.loc[ind]['sequence'].values
    if len(seqs) == 0:
        continue
    if i % (n_splits // 10) == 0:
        print(i)
    struct_X, _ = chimera_tools.make_contact_X(seqs, sample_space, contacts, contact_terms=struct_terms)
    seq_X, _ = chimera_tools.make_sequence_X(seqs,
                                             sample_space=sample_space,
                                             sequence_terms=seq_terms)
    all_X = np.concatenate([seq_X, struct_X], axis=1)
    for clf, fname in zip(clfs, fnames):
        preds = pd.DataFrame(index=df.loc[ind]['name'].values)
        if 'bin' in fname:
            pi, mu, var = clf.predict(all_X)
            preds['pi'] = pi
        else:
            mu, var = clf.predict(all_X)
        var = np.diag(var)
        preds['mu'] = mu
        preds['var'] = var
        with open('../outputs/' + fname + '.txt', 'a') as f:
            preds.to_csv(f, header=False)

0
100
200
300
400
500
600
700
800
900
1000
CPU times: user 7h 41min 48s, sys: 15min 36s, total: 7h 57min 24s
Wall time: 8h 6min


In [15]:
with open('../inputs/GFP_data.pkl', 'rb') as f:
    X, y = pickle.load(f)

def train_and_save(X, y, fname, mtype):
    k = gpkernel.PolynomialKernel(2)
    clf = mtype(k, guesses=None)
    clf.fit(X, y)
    clf.dump(fname)
    return clf

task = 'GFP_above_parent'
mtype = gpmodel.GPClassifier
fname = '../outputs/2' + task + '.pkl'

clf = train_and_save(X, y, fname, mtype)
print(fname)
print(clf.hypers, clf.ML)

../outputs/2GFP_above_parent.pkl
[ 0.36061643  0.04737465] [ 117.78582814]


In [21]:
%%time
with open('../outputs/' + fname + '.txt', 'w') as f:
    f.write('name,p,mu,var\n')
    
df = pd.read_csv('../inputs/all_lit_chimeras_gaps.txt', index_col=0)
with open('../inputs/lit_alignment_and_contacts.pkl', 'rb') as f:
    ss, contacts = pickle.load(f)
amino_acids = ('G', 'A', 'L', 'M', 'F', 'W', 'K', 'Q', 'E', 'S',
       'P', 'V', 'I', 'C', 'Y', 'H', 'R', 'N', 'D', 'T', '-')
sample_space = [amino_acids for _ in ss]
n_splits = 1000
n_per = len(df.index) // n_splits
inds = [df.index[n * n_per: (n+1) * n_per]
        for n in range(n_splits)]
inds.append(df.index[n_splits * n_per::])
seq_terms = chimera_tools.make_sequence_terms(sample_space)
struct_terms = chimera_tools.contacting_terms(sample_space, contacts)
all_terms = seq_terms + struct_terms


for i, ind in enumerate(inds):
    seqs = df.loc[ind]['sequence'].values
    if len(seqs) == 0:
        continue
    if i % (n_splits // 10) == 0:
        print(i)
    struct_X, _ = chimera_tools.make_contact_X(seqs, sample_space, contacts, contact_terms=struct_terms)
    seq_X, _ = chimera_tools.make_sequence_X(seqs,
                                             sample_space=sample_space,
                                             sequence_terms=seq_terms)
    all_X = np.concatenate([seq_X, struct_X], axis=1)
    
    
    preds = pd.DataFrame(index=df.loc[ind]['name'].values)
    pi, mu, var = clf.predict(all_X)
    preds['pi'] = pi
    var = np.diag(var)
    preds['mu'] = mu
    preds['var'] = var
    with open('../outputs/' + fname + '.txt', 'a') as f:
        preds.to_csv(f, header=False)

0
100
200
300
400
500
600
700
800
900
1000
CPU times: user 1h, sys: 16min 42s, total: 1h 16min 42s
Wall time: 1h 7min 57s
